# SSDの学習と検証
- 本ファイルでは、SSDの学習と検証の実施を行います。手元のマシンで動作を確認後、AWSのGPUマシンで計算します。
- p2.xlargeで約6時間かかります。

## Library

In [1]:
# パッケージのimport
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [3]:
# 乱数シードを設定
def something_seed(seed=1234):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

seed = 1234
something_seed(seed)

In [4]:
# cpu or gpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('使用デバイス：　', device)

使用デバイス：　 cpu


## DataLoaderの作成

In [6]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn

# ファイルパスのリストを取得
rootpath = './data/VOCdevkit/VOC2012/'
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

# Datasetを作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 
               'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 
               'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)   # BGRの平均値
input_size = 300  # 画像のサイズを300x300にする

train_dataset = VOCDataset(train_img_list, train_anno_list, phase='train', transform=DataTransform(input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase='val', transform=DataTransform(input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

# DataLoaderを作成
batch_size = 32
train_dataloader = data.DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=od_collate_fn)  # 画像毎に物体検出数が異なるため
val_dataloader = data.DataLoader(val_dataset, batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書オブジェクトへ
dataloaders_dict = {
    'train' : train_dataloader,
    'val' : val_dataloader
}

## ネットワークモデルの作成

In [19]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes' : 21,                                                                   #　背景クラスを含めた合計クラス数
    'input_size' : 300,                                                                     # 画像の入力サイズ
    'bbox_aspect_num' : [4, 6, 6, 6, 4, 4],                                  # 出力するDBoxのアスペクト比の種類
    'feature_maps' : [38, 19, 10, 5, 3, 1],                                   # 各source（feature map）の画像サイズ　
    'steps' : [8, 16, 32, 64, 100, 300],                                        # DBOXの大きさを決める
    'min_sizes' : [30, 60, 111, 162, 213, 264],                         # 
    'max_sizes' : [60, 111, 162, 213, 264, 315],
    'aspect_ratios' : [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# ネットワークモデル
net = SSD(phase='train', cfg=ssd_cfg)

# SSDの重みの初期値を設定
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)  # vgg忘れに注意　　つけないと全てのネットワークに対して重み入れようとしてエラーに

# ssdのvgg以外のnetworkの重みはHeの初期値（sqrt(2/n)）で初期化
# Heの初期値は画像でReLUを活性化カンスに使うときに使用する
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # bias項がある場合
            nn.init.constant_(m.bias, 0.0)
            
# Heの重みを適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)
# applyのこういう使い方かっこいい

# GPUが使えるかを確認
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('使用デバイス：　', device)

print('ネットワーク設定完了：　学習済みの重みをロードしました')

使用デバイス：　 cpu
ネットワーク設定完了：　学習済みの重みをロードしました


In [12]:
# SSDモデルの確認
# self,vgg, self.extrasというふうにインスタンス変数でModuleListをもっている
net.vgg

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

In [13]:
net.extras

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)

In [14]:
net.loc

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [15]:
net.conf

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [16]:
net

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

## 損失関数、最適化手法の設定


In [20]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)


## 学習と検証
モデルを学習させる関数を作成する。kerasで言うところのfitみたいなもの

In [25]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    # GPUが使えるか
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('使用デバイス：　', device)
    
    # ネットワークをGPUへ
    net.to(device)
    
    # ネットワークがある程度固定できれば高速化させる
    torch.backends.cudnn.benchmark = True
    
    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0   # epochの損失和
    epoch_val_loss = 0.0      # epochの損失和
    logs = []
    
    # epochのループ
    # この中で trainとvalを行う
    for epoch in range(num_epochs+1):
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        
        print('-------------------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------------------')
        
        #  epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('  (train)  ')
            else:
                # 検証は１０学習に付き１回行う
                if ((epoch+1)%10) == 0:
                    net.eval()
                    print('-------------------------')
                    print('  (val)  ')
                else:
                    continue
                    
            # データローダーからminibatchずつ取り出すグループ
            for images, targets in dataloaders_dict[phase]:   # train, valどっちから取り出すのか指定
                # GPUに送る
                images = images.to(device)
                targets = [anno.to(device) for anno in targets]  # リストの各要素のテンソルをGPUへ
                # そのまま送ると不都合あるのかな？
                
                # optimizerを初期化
                optimizer.zero_grad()
                
                # 順伝播計算
                with torch.set_grad_enable(phase=='train'):
                    outputs = net(images)
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c
                    
                    # 学習時
                    if phase == 'train':
                        loss.backward()  # 勾配求まる
                        
                        # 勾配が大きくなりすぎると学習が不安定になるのでclipで最大でも２にする
                        nn.utils.clip_grad_value(net.parameters(), clip_value=2.0)
                        optimizer.step()
                        
                        # 10回に１回lossを表示
                        if (iteraton%10 == 0):
                            t_iter_finish =  time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(iteration, loss.item(), duration))
                            t_iter_start = time.time()   # iterの時間計測再開
                            
                        epoch_train_loss += loss.item()
                        iteration += 1
                        
                    # 検証時
                    else:
                        epoch_val_loss += loss
                        
        # epochのphaseごとのlossと正解率　　　表示させたら０．０に戻す
        t_epoch_finish =  time.time()
        print('-------------------------')
        print('epoch {} || EPOCH_TRAIN_Loss : {:.4f} || EPOCH_VAL_Loss : {:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer : {:.4f} sec'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()  # epochの時間計測再開
        
        # logを保存
        log_epoch = {'epoch': epoch+1, 
                    'train_loss': epoch_train_loss,
                    'val_loss' : epoch_val_loss}
        logs.append(log_epoch)   # リストにdictを追加していく
        df = pd.DataFrame(logs)
        df.to_csv('log_output.csv')  
        # 毎epoch、log保存していったら結構な数になるじゃないか？　-> 
        # ずっと同じlog_output.csvに保存しているから、フィアルが増えるわけではなくて上書きされてく
        
        # 保存し終わったところでlossを初期化
        epoch_train_loss = 0.0   # epochの損失和
        epoch_val_loss = 0.0      # epochの損失和
        
        # ネットワークを保存する  全部で５回
        if ((epoch+1)%10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_'+str(epoch+1)+'.pth')
            
        

In [ ]:
# 学習・検証をする
# これはaws上で回す
num_epochs = 50
train_model(net, dataloader_dict, criterion, optimizer, num_epochs)